In [ ]:
from flask import Flask, render_template, request
import sqlite3
import math
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text


conn = sqlite3.connect('DOSM.db')

In [ ]:
app = Flask(__name__)
conn = sqlite3.connect('DOSM.db')


In [ ]:
# render home.html
@app.route('/')
def home():
    return render_template('home.html')



In [ ]:
from flask import Flask, render_template, request
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import text

@app.route('/result', methods=['GET','POST'])
def result():
    
    # Retrieve user input from the form
    item_category = request.form.get('item_category')
    month = request.form.get('month')
    year = request.form.get('year')
    day = request.form.get('day')
    SQLALCHEMY_ECHO=True
    
    print(f"item_category: {item_category}")
    print(f"month: {month}")
    print(f"year: {year}")
    print(f"day: {day}")
    print(request.method)

    # Split the input values if there are multiple selections
    item_category = item_category.split(",") if item_category else []
    month_list = month.split(",") if month else []
    year_list = year.split(",") if year else []
    day_list = day.split(",") if day else []

    # Connect to the database
    conn = sqlite3.connect('DOSM.db')
    c = conn.cursor()

    # Set the base query
    query = "SELECT item, price, unit, item_group, item_category, premise, premise_type, state, district, address, date " \
            "FROM Price " \
            "JOIN Item ON Price.item_code = Item.item_code " \
            "JOIN Premise ON Price.premise_code = Premise.premise_code " \
            "WHERE 1 = 1 "

    # Add filters for item category, month, year, and day
    query_params = []
    if item_category:
        # Use the IN operator to filter by multiple categories
        query += f"AND item_category IN ({','.join(['?' for _ in range(len(item_category))])}) "
        query_params.extend(item_category)
    if month:
        if len(month_list) == 1:
            query += "AND month = ? "
            query_params.append(month)
        elif len(month_list) == 2:
            query += "AND month BETWEEN ? AND ? "
            query_params.extend(month_list)
    if year:
        if len(year_list) == 1:
            query += "AND year = ? "
            query_params.append(year)
        elif len(year_list) == 2:
            query += "AND year BETWEEN ? AND ? "
            query_params.extend(year_list)
    if day:
        if len(day_list) == 1:
            query += "AND day = ? "
            query_params.append(day)
        elif len(day_list) == 2:
            query += "AND day BETWEEN ? AND ? "
            query_params.extend(day_list)

    # Get the count of total records for pagination
    c.execute(f"SELECT COUNT(*) FROM ({query})", query_params)
    total_count = c.fetchone()[0]
            
    # Execute the query with the parameters
    result = []
    page = request.args.get('page', 1, type=int)
    per_page = request.args.get('per_page', 50, type=int)
    query += "ORDER BY date DESC"
    query += f" LIMIT {per_page} OFFSET {(page-1)*per_page}"
    
    total_pages = math.ceil(total_count / per_page)

    c.execute(query, query_params)
    result = c.fetchall()
    
    print(f"query: {query}")
    print(f"query_params:{query_params}")
    print(f"total_count:{total_count}")
    print(f"page:{page}")
    print(f"per_page:{per_page}")
    
    # Close the database connection
    conn.close()

    # Render the result.html template with the query result and pagination information
    return render_template('result.html', result=result, page=page, per_page=per_page, total_count=total_count, total_pages=total_pages)



In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)